In [ ]:
import csv
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import radioactivedecay as rd
from pypost.codes.melcor import MELCOR

# =======================
# INPUT FROM MELCOR (same variable structure)
# =======================

# --- Adjust here the file and the CVHs you want to use ---
filename = "MC_Step.ptf"
cv_list = [110, 160, 200, 305, 1, 600, 201]

# --- Adjust here the CFVALU range that previously came from JSON ---
start_number = 3000     # <-- set your real start
end_number   = 3062     # <-- set your real end

# Open MELCOR file
index = MELCOR.openPlotFile(filename)

# Take time values from the first reference variable (as requested)
primer_cv = cv_list[0]
var_ref = f"CVH-VOLLIQ_{primer_cv}"
data_ref = np.array(MELCOR.getData(index, var_ref))

t_ini_raw = float(data_ref[0, 0])
t_fin_raw = float(data_ref[-1, 0])

# +1 s and truncated (remove decimals)
start_time = int(t_ini_raw + 1)
end_time   = int(t_fin_raw + 1)

# =======================
# Calculation of V_values (keys 1..N) and index->CVH map
# =======================
V_values = {}                 # dict[int -> float] in cm³; keys 1..N
cv_index_map = {}             # dict[int -> int]  index->CVH (for info only)

for idx, cv in enumerate(cv_list, start=1):   # idx = 1..N
    cv_index_map[idx] = int(cv)

    var_liq = f"CVH-VOLLIQ_{cv}"
    var_vap = f"CVH-VOLVAP_{cv}"

    data_liq = np.array(MELCOR.getData(index, var_liq))
    data_vap = np.array(MELCOR.getData(index, var_vap))

    t_liq = data_liq[:, 0]
    v_liq = data_liq[:, 1]
    t_vap = data_vap[:, 0]
    v_vap = data_vap[:, 1]

    # Align time arrays if needed
    if (data_liq.shape[0] != data_vap.shape[0]) or (not np.allclose(t_liq, t_vap)):
        v_vap = np.interp(t_liq, t_vap, v_vap)

    total_m3 = v_liq + v_vap
    avg_m3   = float(np.mean(total_m3))
    avg_cm3  = avg_m3 * 1_000_000.0  # m³ -> cm³

    # Key = sequential index (1..N), not the CVH number
    V_values[idx] = avg_cm3

# Build CFVALU and batches (identical to your structure)
mass_data_lines = [f"CFVALU_{i}" for i in range(start_number, end_number + 1)]
batches = [mass_data_lines[i:i + 9] for i in range(0, len(mass_data_lines), 9)]
num_batches = len(batches)

# Messages (same organization and variables; only data source changes)
print(f"\n[INFO] Parameters loaded from MELCOR (no JSON)")
print(f" - Control functions from {start_number} to {end_number}")
print(f" - Simulation time: {start_time}s to {end_time}s")
print(f" - Number of control volumes (batches): {num_batches}")
print(f" - Volumes (keys 1..N, cm³): {V_values}")

# (Optional) Show index -> CVH mapping
print(f" - Index->CVH map: {cv_index_map}")





In [ ]:
# ================================
# OUTPUTS Y RADIONUCLIDE CLASSES
# ================================
output_directory_1 = "proportions"
recalculate_proportions = False

if not os.path.exists(output_directory_1):
    os.makedirs(output_directory_1)
    recalculate_proportions = True
else:
    expected_files = [f"class_{i}_proportions.csv" for i in range(1, 10)]
    existing_files = os.listdir(output_directory_1)
    if not all(f in existing_files for f in expected_files):
        recalculate_proportions = True

with open("class_inventories.json", "r") as f:
    class_inventories = json.load(f)

class_inventories = {int(k): v for k, v in class_inventories.items()}        

time_steps = np.arange(start_time, end_time, 1)
all_class_proportions = {}


In [ ]:
import csv
import pandas as pd


# ==========================================
# CALCULATION OR LOADING OF PROPORTIONS
# ==========================================
output_directory_1 = "proportions"
os.makedirs(output_directory_1, exist_ok=True)
expected_files = [f"class_{i}_proportions.csv" for i in range(1, 10)]
recalc = not all(f in os.listdir(output_directory_1) for f in expected_files)

all_class_proportions = {}

if recalc:
    for class_num, inv_props in class_inventories.items():
        print(f"Calculating proportions for class {class_num}...")
        inv = rd.Inventory({}, "kg")
        # initialize 1 kg total distributed according to inventory proportions
        inv += rd.Inventory({iso: 1.0 * prop for iso, prop in inv_props.items()}, "kg")
        proportions = {iso: np.zeros(len(time_steps)) for iso in inv_props}
        last_t = time_steps[0]
        for i, t in enumerate(time_steps):
            inv = inv.decay(t - last_t, 's') if i > 0 else inv
            last_t = t
            masses = inv.masses("kg")
            total = sum(masses.values())
            for iso in proportions:
                proportions[iso][i] = max(masses.get(iso, 0), 0) / total if total > 0 else 0

        all_class_proportions[class_num] = proportions

        # save CSV
        csv_path = os.path.join(output_directory_1, f"class_{class_num}_proportions.csv")
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Time (s)"] + list(proportions.keys()))
            for i, t in enumerate(time_steps):
                writer.writerow([t] + [proportions[iso][i] for iso in proportions])

        # plot results
        plt.figure(figsize=(8, 5))
        for iso, arr in proportions.items():
            plt.plot(time_steps, arr, label=iso)
        plt.legend(); plt.grid(True)
        plt.savefig(os.path.join(output_directory_1, f"class_{class_num}_proportions.png"))
        plt.close()

else:
    for class_num in range(1, 10):
        csv_path = os.path.join(output_directory_1, f"class_{class_num}_proportions.csv")
        df = pd.read_csv(csv_path)
        isotopes = df.columns.tolist()[1:]
        all_class_proportions[class_num] = {
            iso: df[iso].values for iso in isotopes
        }
    print("Proportions successfully loaded.")



In [ ]:
import os
import csv
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from pypost.codes.melcor import MELCOR

# ==========================================
# MODULE: CALCULATION OR LOADING OF MASSES (all batches)
# ==========================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Masses ===")

    # 1) Output directory, create if necessary
    output_directory = f"output_batch_{batch_num}"
    os.makedirs(output_directory, exist_ok=True)

    # 2) Do the mass CSV files already exist?
    expected = [f"class_{i}_masses.csv" for i in range(1, 10)]
    recalc = not all(fn in os.listdir(output_directory) for fn in expected)

    all_class_masses = {}

    if recalc:
        print("[INFO] Calculating masses from MELCOR…")
        batch = batches[batch_num - 1]               # the 9 variables of this batch

        # Loop through each of the 9 classes
        for class_num, var in enumerate(batch, start=1):
            print(f"  Class {class_num}: variable '{var}'")
            # 2.1) Read from MELCOR
            idx = MELCOR.openPlotFile("MC_Step.ptf")
            data = np.array(MELCOR.getData(idx, var))
            t, M = data[:, 0], data[:, 1]

            # 2.2) Filter and make uniform
            mask = (t >= start_time) & (t <= end_time)
            uni_t = np.arange(start_time, end_time, 1)
            uni_M = interp1d(t[mask], M[mask],
                             kind='previous',
                             fill_value="extrapolate")(uni_t)

            # 2.3) Distribute according to proportions
            props = all_class_proportions[class_num]
            masses = {iso: props[iso] * uni_M for iso in props}
            all_class_masses[class_num] = masses

            # 2.4) Save to CSV
            csv_path = os.path.join(output_directory,
                                    f"class_{class_num}_masses.csv")
            with open(csv_path, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(masses.keys()))
                for i, t0 in enumerate(uni_t):
                    w.writerow([t0] + [masses[iso][i] for iso in masses])

            # 2.5) Plot results
            plt.figure(figsize=(8, 5))
            for iso, arr in masses.items():
                plt.plot(uni_t, arr, label=iso)
            plt.xlabel("Time (s)")
            plt.ylabel("Mass (kg)")
            plt.title(f"Batch {batch_num} – Class {class_num} Masses")
            plt.legend(); plt.grid(True)
            plt.savefig(os.path.join(output_directory,
                                     f"class_{class_num}_masses.png"))
            plt.close()

        print(f"[Batch {batch_num}] Masses calculated and saved.")

    else:
        print("[INFO] Loading masses from CSV…")
        for class_num in range(1, 10):
            csv_path = os.path.join(output_directory,
                                    f"class_{class_num}_masses.csv")
            df = pd.read_csv(csv_path)
            isotopes = df.columns.tolist()[1:]
            all_class_masses[class_num] = {
                iso: df[iso].values for iso in isotopes
            }
        print(f"[Batch {batch_num}] Masses successfully reloaded.")

    # Now `all_class_masses` is ready for this batch_num
    # Quick check:
    print(f"  Class 1 – first mass value for '{list(all_class_masses[1].keys())[0]}' =",
          all_class_masses[1][list(all_class_masses[1].keys())[0]][0])




In [ ]:

import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import radioactivedecay as rd

# ==========================================
# MODULE: CALCULATION OR LOADING OF ACTIVITIES (all batches)
# ==========================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Activities ===")

    # 1) Output directory and subfolder for plots
    output_directory = f"output_batch_{batch_num}"
    dose_plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(dose_plots_dir, exist_ok=True)

    # 2) Reload MASSES from CSV
    all_class_masses = {}
    for class_num in range(1, 10):
        mass_csv = os.path.join(output_directory, f"class_{class_num}_masses.csv")
        df_mass = pd.read_csv(mass_csv)
        isotopes = df_mass.columns.tolist()[1:]
        all_class_masses[class_num] = {iso: df_mass[iso].values for iso in isotopes}

    # 3) Check for existing ACTIVITY CSV files
    expected = [f"class_{i}_activities.csv" for i in range(1, 10)]
    recalc_act = not all(fn in os.listdir(output_directory) for fn in expected)

    all_class_activities = {}

    if recalc_act:
        print("[INFO] Calculating activities from masses…")
        for class_num, isotope_masses in all_class_masses.items():
            print(f"  Class {class_num}")
            # Prepare the activity dictionary (Ci)
            acts = {iso: np.zeros(len(time_steps)) for iso in isotope_masses}

            for idx, t in enumerate(time_steps):
                inv = rd.Inventory({}, "kg")
                for iso, mass_arr in isotope_masses.items():
                    m = mass_arr[idx]
                    if m > 0:
                        inv += rd.Inventory({iso: m}, "kg")
                bq_dict = inv.activities("Bq")
                for iso, bq in bq_dict.items():
                    acts[iso][idx] = bq / 3.7e10  # Convert Bq → Ci

            all_class_activities[class_num] = acts

            # Save CSV
            csv_path = os.path.join(output_directory, f"class_{class_num}_activities.csv")
            with open(csv_path, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(acts.keys()))
                for i, t in enumerate(time_steps):
                    w.writerow([t] + [acts[iso][i] for iso in acts])

            # Plot
            plt.figure(figsize=(8, 5))
            for iso, arr in acts.items():
                plt.plot(time_steps, arr, label=iso)
            plt.xlabel("Time (s)")
            plt.ylabel("Activity (Ci)")
            plt.title(f"Batch {batch_num} – Class {class_num} Activities")
            plt.legend(); plt.grid(True)
            plt.savefig(os.path.join(output_directory, f"class_{class_num}_activities.png"))
            plt.close()

        print(f"[Batch {batch_num}] Activities calculated and saved.")

    else:
        print("[INFO] Loading activities from CSV…")
        for class_num in range(1, 10):
            csv_path = os.path.join(output_directory, f"class_{class_num}_activities.csv")
            df_act = pd.read_csv(csv_path)
            isotopes = df_act.columns.tolist()[1:]
            all_class_activities[class_num] = {
                iso: df_act[iso].values for iso in isotopes
            }
        print(f"[Batch {batch_num}] Activities successfully reloaded.")

    # Now you can use all_class_activities[batch_num]...





In [ ]:
import json

with open("energies_data.json", "r") as f:
    data = json.load(f)

gamma_energies = data["gamma_energies"]
beta_emitter_energies = data["beta_emitter_energies"]


In [ ]:
import os
import csv
import numpy as np
import pandas as pd

# ==========================================
# MODULE: ADD BREMSSTRAHLUNG CONTRIBUTION
# ==========================================
# Assumes gamma_energies and beta_emitter_energies are already defined elsewhere

Z_air = 7.3  # effective atomic number of air

for iso, E_beta in beta_emitter_energies.items():
    # Add bremsstrahlung energy (in MeV)
    E_brem = 1.33e-4 * Z_air * E_beta**2
    gamma_energies[iso] = gamma_energies.get(iso, 0) + E_brem

print("[INFO] Bremsstrahlung energy added to gamma_energies for all isotopes.")

# ==========================================
# SKIPPING AVERAGE GAMMA ENERGY CALCULATION
# ==========================================
# This block previously calculated the average gamma energy per class and per time step.
# It has been intentionally removed since only the bremsstrahlung correction is needed.




In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import json

# ==================================
# LOAD GAMMA ENERGIES (with bremsstrahlung already added)
# ==================================
with open("energies_data.json", "r") as f:
    energies_data = json.load(f)

gamma_energies = energies_data["gamma_energies"]

# ==================================
# CONSTANTS FOR DOSE RATE CALCULATION
# ==================================

# These are updated dynamically per batch in the next cell
density = 0.001  # g/cm³

# ==================================
# MASS ATTENUATION AND ABSORPTION FUNCTIONS
# ==================================

def mass_att(E, a1=-0.037274, b1=0.101714, c1=-0.274123):
    """Mass attenuation coefficient as a function of photon energy E (MeV)."""
    if E <= 0:
        return 0
    return a1 + b1 * E**c1

def mass_ab(E, a2=0, b2=-5.2588e-4, c2=-5.2077e-3, d=2.8172e-2, e=-1.7809e-2):
    """Mass absorption coefficient as a function of photon energy E (MeV)."""
    if E <= 0:
        return 0
    return a2 + b2 * E + c2 * (math.log(E))**2 + d * math.sqrt(E) + e * math.log(E)

def calc_mu_m(E):
    """Compute μm depending on photon energy E (keV)."""
    if 0 <= E <= 200:
        B0, B1, B2, B3 = 0.29839, -0.00269, 1.67948E-5, -3.75963E-8
        mu_m = B3 * E**3 + B2 * E**2 + B1 * E + B0
    elif E > 200:
        B0, B1, B2 = 0.13556, -9.10106E-5, 2.39846E-8
        mu_m = B2 * E**2 + B1 * E + B0
    else:
        return 0
    return mu_m

# ==================================
# CIRCULAR BASE METHOD FOR GAMMA FLUX
# ==================================

def gamma_flux_circular_base(activity, R, h, mass_att_func, E):
    """Computes gamma flux using the circular base method."""
    if E <= 0 or activity <= 0:
        return 0.0
    mu = mass_att_func(E)
    if mu <= 0:
        return 0.0

    r = np.linspace(0, R, 10)
    theta = np.linspace(0, np.arctan(R / h), 10)
    R_grid, Theta_grid = np.meshgrid(r, theta)
    path_length = np.sqrt(R_grid**2 + h**2)
    b1 = mu * path_length
    E1_term = sp.exp1(b1)
    E1_sec_term = sp.exp1(b1 / np.cos(Theta_grid))
    integrand = (((3.7e10) * activity) / (2 * np.pi * R**2)) * (E1_term - E1_sec_term)
    result = np.trapz(np.trapz(integrand * R_grid, r, axis=0), theta)
    return result

# ==================================
# PLOTTING FUNCTIONS FOR DOSE RATES
# ==================================

def plot_dose_rates(dose_rates, title, filename, out_dir):
    """Plot dose rate evolution for multiple isotopes."""
    plt.figure(figsize=(10, 6))
    for iso, dr in dose_rates.items():
        plt.plot(time_steps, dr, label=f"{iso} Dose Rate (rads/h)")
    plt.xlabel("Time (s)")
    plt.ylabel("Dose Rate (rads/h)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(out_dir, filename))
    plt.close()

def plot_accumulated_dose(total_dose, title, filename, out_dir):
    """Plot total accumulated dose as a function of time."""
    plt.figure(figsize=(10, 6))
    plt.plot(time_steps, total_dose, label="Total Accumulated Dose (rads)")
    plt.xlabel("Time (s)")
    plt.ylabel("Accumulated Dose (rads)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(out_dir, filename))
    plt.close()


In [ ]:
import os
import csv
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Sphere Hypothesis)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Dose Rates (Sphere) ===")

    # 1) Output directories and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        act_csv = os.path.join(output_directory, f"class_{class_num}_activities.csv")
        df_act = pd.read_csv(act_csv)
        isotopes = df_act.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isotopes}

    # 3) Geometric parameters
    R_sphere = ((3 * V) / (4 * np.pi)) ** (1 / 3)
    density = 0.001  # g/cm³

    # 4) Check for per-class CSV files
    class_csvs = [f"dose_rate_sphere_class_{i}.csv" for i in range(1, 10)]
    have_class = all(fn in os.listdir(plots_dir) for fn in class_csvs)

    # 5) Check for total accumulated CSV
    total_csv = "total_accumulated_dose_sphere_all_classes.csv"
    have_total = total_csv in os.listdir(plots_dir)

    total_accumulated = np.zeros(len(time_steps))

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if not have_class:
        print("[INFO] Missing per-class dose rate CSVs → recalculating everything…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            total_per_time = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                step_sum = 0.0
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue
                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    flux = (3.7e10) * (A / (V * density * mu)) * (1 - np.exp(-density * R_sphere * mu))
                    dr = 5.77e-5 * flux * E * sigma
                    dose_rates[iso][i] = dr
                    step_sum += dr

                total_per_time[i] = step_sum / 3600.0  # rads/h

            # Accumulate dose
            acc = 0.0
            for i in range(len(time_steps)):
                acc += total_per_time[i]
                total_accumulated[i] += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_sphere_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["Time (s)"] + list(dose_rates.keys()))
                for j, tt in enumerate(time_steps):
                    writer.writerow([tt] + [dose_rates[iso][j] for iso in dose_rates])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates,
                f"Dose Rates (Sphere) - Class {class_num}",
                f"dose_rates_sphere_class_{class_num}.png",
                out_dir=plots_dir
            )

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs already exist.")
        for class_num in range(1, 10):
            df_dr = pd.read_csv(os.path.join(plots_dir, f"dose_rate_sphere_class_{class_num}.csv"))
            isot = df_dr.columns.tolist()[1:]
            drs = {iso: df_dr[iso].values for iso in isot}

            acc = 0.0
            for i in range(len(time_steps)):
                step = sum(drs[iso][i] for iso in isot) / 3600.0
                acc += step
                total_accumulated[i] += acc

    # 6) Save or reload total accumulated CSV
    total_path = os.path.join(plots_dir, total_csv)
    if not have_total:
        print("[INFO] Missing total accumulated CSV → generating now…")
        with open(total_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists. Reloading values…")
        df_tot = pd.read_csv(total_path)
        total_accumulated = df_tot["Accumulated_Dose (rads)"].values

    # 7) Plot total accumulated dose
    plot_accumulated_dose(
        total_accumulated,
        f"Total Accumulated Dose (Sphere) - Batch {batch_num}",
        f"total_accumulated_dose_sphere_batch_{batch_num}.png",
        out_dir=plots_dir
    )




In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import scipy.special as sp

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Circular Base Method)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Circular Base Method ===")

    # 1) Output directory and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        act_csv = os.path.join(output_directory, f"class_{class_num}_activities.csv")
        df_act = pd.read_csv(act_csv)
        isotopes = df_act.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isotopes}

    # 3) Parameters and total accumulator
    total_accumulated_csv = f"total_accumulated_dose_cb_batch_{batch_num}.csv"
    total_accumulated_path = os.path.join(plots_dir, total_accumulated_csv)

    total_accumulated_dose_cb = np.zeros(len(time_steps))
    h_base = 25  # cm (height of source)
    R_base = 15  # cm (radius of circular base)

    # 4) Check if per-class recalculation is needed
    expected = [f"dose_rate_circular_base_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSV files → calculating dose rates (Circular Base)…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates_cb = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            total_per_timestep = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                sum_step = 0.0
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    # Compute gamma flux using the circular base geometry
                    flux = gamma_flux_circular_base(A, R_base, h_base, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma
                    if np.isnan(dr):
                        continue

                    dose_rates_cb[iso][i] = dr
                    sum_step += dr / 3600.0  # convert to rads/h

                total_per_timestep[i] = sum_step

            # Accumulate dose per class
            acc = 0.0
            for i in range(len(time_steps)):
                acc += total_per_timestep[i]
                total_accumulated_dose_cb[i] += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_circular_base_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(dose_rates_cb.keys()))
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_cb[iso][j] for iso in isotopes])

            # Plot per-class dose rate
            plot_dose_rates(
                dose_rates_cb,
                f"Dose Rates (Circular Base) - Class {class_num}",
                f"dose_rates_circular_base_class_{class_num}.png",
                out_dir=plots_dir
            )

        print(f"[Batch {batch_num}] Per-class dose rates calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSV files exist → reloading dose rates from CSV…")

        for class_num in range(1, 10):
            df_cb = pd.read_csv(os.path.join(plots_dir, f"dose_rate_circular_base_class_{class_num}.csv"))
            isot = df_cb.columns.tolist()[1:]
            drs = {iso: df_cb[iso].values for iso in isot}

            acc = 0.0
            for i in range(len(time_steps)):
                step = sum(drs[iso][i] for iso in isot) / 3600.0
                acc += step
                total_accumulated_dose_cb[i] += acc

        print(f"[Batch {batch_num}] Dose rates successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_accumulated_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_accumulated_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated_dose_cb):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_accumulated_path)
        total_accumulated_dose_cb = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        total_accumulated_dose_cb,
        f"Total Accumulated Dose (Circular Base) - Batch {batch_num}",
        f"total_accumulated_dose_circular_base_batch_{batch_num}.png",
        out_dir=plots_dir
    )




In [ ]:
import os
import csv
import math
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Circular Base + Shielding)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Circular Base + Shielding ===")

    # 1) Output directory and control volume
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)
    V = V_values[batch_num]

    # 2) Reload ACTIVITIES (gamma energies come from JSON)
    all_class_activities = {}
    for class_num in range(1, 10):
        df = pd.read_csv(os.path.join(output_directory, f"class_{class_num}_activities.csv"))
        isos = df.columns.tolist()[1:]
        all_class_activities[class_num] = {iso: df[iso].values for iso in isos}

    # 3) Geometrical parameters, material properties, and total accumulator
    total_acc_csv = f"total_accumulated_dose_shield_batch_{batch_num}.csv"
    total_acc_path = os.path.join(plots_dir, total_acc_csv)

    total_accumulated_dose_shield = np.zeros(len(time_steps))
    cylinder_radius = 15   # cm
    cylinder_height = 80   # cm
    R_shield = math.sqrt(2 * cylinder_radius * cylinder_height)
    density_polymer = 0.92  # g/cm³
    shield_thickness = 3    # cm

    def linear_att(E_keV, rho):
        """Linear attenuation coefficient as μ = μm(E) * ρ"""
        return calc_mu_m(E_keV) * rho

    # 4) Check if per-class recalculation is needed
    expected = [f"dose_rate_shield_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSVs → calculating Shielding…")

        for class_num, isotopes in class_inventories.items():
            acts = all_class_activities[class_num]
            dose_rates_shield = {iso: np.zeros(len(time_steps)) for iso in isotopes}
            timestep_dose = np.zeros(len(time_steps))

            for i, t in enumerate(time_steps):
                for iso in isotopes:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    mu = mass_att(E)
                    sigma = max(1e-4, mass_ab(E))
                    if mu == 0:
                        continue

                    # Compute unshielded flux using circular base geometry
                    flux = gamma_flux_circular_base(A, R_shield, shield_thickness, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma

                    # Apply exponential attenuation through the shield
                    dr_sh = dr * np.exp(-linear_att(E * 1000, density_polymer) * shield_thickness)

                    if np.isnan(dr_sh):
                        continue

                    dose_rates_shield[iso][i] = dr_sh
                    timestep_dose[i] += dr_sh / 3600.0  # convert to rads/h

            # Accumulate per class
            acc = np.cumsum(timestep_dose)
            total_accumulated_dose_shield += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_shield_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + list(dose_rates_shield.keys()))
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_shield[iso][j] for iso in isotopes])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates_shield,
                f"Dose Rates (Shielding) - Class {class_num}",
                f"dose_rate_shield_class_{class_num}.png",
                out_dir=plots_dir
            )
            print(f"  Class {class_num} → final accumulated dose: {acc[-1]:.2f} rads")

        print(f"[Batch {batch_num}] Shielding calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs exist → reloading Shielding from CSV…")

        for class_num in range(1, 10):
            df = pd.read_csv(os.path.join(plots_dir, f"dose_rate_shield_class_{class_num}.csv"))
            isos = df.columns.tolist()[1:]
            drs = {iso: df[iso].values for iso in isos}

            # Accumulate
            step = np.array([sum(drs[iso][i] for iso in isos) / 3600.0 for i in range(len(time_steps))])
            total_accumulated_dose_shield += np.cumsum(step)

        print(f"[Batch {batch_num}] Shielding successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_acc_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_acc_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, total_accumulated_dose_shield):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_acc_path)
        total_accumulated_dose_shield = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        total_accumulated_dose_shield,
        f"Total Accumulated Dose (Shielding) - Batch {batch_num}",
        f"total_accumulated_dose_shield_batch_{batch_num}.png",
        out_dir=plots_dir
    )




In [ ]:
import os
import csv
import math
import numpy as np
import pandas as pd

# ===============================================
# MODULE: CALCULATION OR LOADING OF DOSE RATE (Point Source Method)
# ===============================================
for batch_num in range(1, len(batches) + 1):
    print(f"\n=== Batch {batch_num} — Point Source Method ===")

    # 1) Output directory and plots
    output_directory = f"output_batch_{batch_num}"
    plots_dir = os.path.join(output_directory, 'dose_rate_plots')
    os.makedirs(plots_dir, exist_ok=True)

    # 2) Reload ACTIVITIES
    all_class_activities = {}
    for class_num in range(1, 10):
        df_act = pd.read_csv(os.path.join(output_directory, f"class_{class_num}_activities.csv"))
        isos = df_act.columns.tolist()[1:]  # skip time column
        all_class_activities[class_num] = {iso: df_act[iso].values for iso in isos}

    # 3) Parameters and total accumulator
    total_acc_csv = f"total_accumulated_dose_point_batch_{batch_num}.csv"
    total_acc_path = os.path.join(plots_dir, total_acc_csv)
    accumulated_dose_point = np.zeros(len(time_steps))
    h_point = 25  # cm (distance from source to receptor)
    density = 0.001  # g/cm³ (air density)

    def gamma_flux_point_source(activity, h, mass_att_func, E):
        """Gamma flux for a point source with attenuation (μ * ρ * h)."""
        mu = mass_att_func(E)
        if mu <= 0:
            return 0.0
        # 3.7e10 converts Ci → disintegrations/s if activity is in Ci
        return (3.7e10 * activity) / (4 * math.pi * h**2) * math.exp(-mu * density * h)

    # 4) Check for per-class CSVs
    expected = [f"dose_rate_point_source_class_{i}.csv" for i in range(1, 10)]
    need_recalc = not all(fn in os.listdir(plots_dir) for fn in expected)

    # ===============================================
    # CALCULATE IF MISSING
    # ===============================================
    if need_recalc:
        print("[INFO] Missing per-class CSVs → calculating Point Source…")

        for class_num, isotopes in class_inventories.items():
            # Extract isotope names if dict
            iso_names = list(isotopes.keys()) if isinstance(isotopes, dict) else list(isotopes)
            acts = all_class_activities[class_num]  # dict: iso -> activity array

            dose_rates_point = {iso: np.zeros(len(time_steps)) for iso in iso_names}
            timestep_dose = np.zeros(len(time_steps))  # sum at each timestep

            for i, t in enumerate(time_steps):
                for iso in iso_names:
                    E = gamma_energies.get(iso, 0.0)
                    if E <= 0:
                        continue

                    A = acts.get(iso, np.zeros(len(time_steps)))[i]
                    if A <= 0:
                        continue

                    sigma = max(1e-4, mass_ab(E))  # effective absorption coefficient
                    flux = gamma_flux_point_source(A, h_point, mass_att, E)
                    dr = 5.77e-5 * flux * E * sigma  # MeV → rads/s conversion

                    if np.isnan(dr):
                        continue

                    dose_rates_point[iso][i] = dr
                    timestep_dose[i] += dr / 3600.0  # convert to rads/h

            # Accumulate over time
            acc = np.cumsum(timestep_dose)
            accumulated_dose_point += acc

            # Save per-class CSV
            out_csv = os.path.join(plots_dir, f"dose_rate_point_source_class_{class_num}.csv")
            with open(out_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)"] + iso_names)
                for j, t in enumerate(time_steps):
                    w.writerow([t] + [dose_rates_point[iso][j] for iso in iso_names])

            # Plot per-class dose rates
            plot_dose_rates(
                dose_rates_point,
                f"Dose Rates (Point Source) - Class {class_num}",
                f"dose_rate_point_source_class_{class_num}.png",
                out_dir=plots_dir
            )

        print(f"[Batch {batch_num}] Point Source dose rates calculated.")

    # ===============================================
    # IF EXISTS, JUST RELOAD
    # ===============================================
    else:
        print("[INFO] Per-class CSVs already exist → reloading Point Source from CSV…")

        for class_num in range(1, 10):
            df_ps = pd.read_csv(os.path.join(plots_dir, f"dose_rate_point_source_class_{class_num}.csv"))
            isos = df_ps.columns.tolist()[1:]
            drs = {iso: df_ps[iso].values for iso in isos}

            steps = np.array([sum(drs[iso][i] for iso in isos) / 3600.0 for i in range(len(time_steps))])
            accumulated_dose_point += np.cumsum(steps)

        print(f"[Batch {batch_num}] Point Source successfully reloaded.")

    # ===============================================
    # SAVE OR RELOAD TOTAL ACCUMULATED DOSE
    # ===============================================
    if not os.path.exists(total_acc_path):
        print("[INFO] Missing total accumulated CSV → generating it now…")
        with open(total_acc_path, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
            for tt, val in zip(time_steps, accumulated_dose_point):
                w.writerow([tt, val])
    else:
        print("[INFO] Total accumulated CSV already exists → reloading…")
        df_tot = pd.read_csv(total_acc_path)
        accumulated_dose_point = df_tot["Accumulated_Dose (rads)"].values

    # ===============================================
    # PLOT TOTAL ACCUMULATED DOSE
    # ===============================================
    plot_accumulated_dose(
        accumulated_dose_point,
        f"Total Accumulated Dose (Point Source) - Batch {batch_num}",
        f"total_accumulated_dose_point_source_batch_{batch_num}.png",
        out_dir=plots_dir
    )

